<a href="https://colab.research.google.com/github/Ele975/LLM_from_scratch/blob/development/LLM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

In [ ]:
# for tokenizer
!pip install tiktoken

# for weights loading
!pip install tensorflow>=2.15.0
!pip install tqdm>=4.66

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00


In [ ]:
import re
import urllib.request
import tiktoken
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import pandas as pd
import torch

from torch.utils.data import Dataset
from pathlib import Path
from matplotlib.ticker import MaxNLocator
from torch.utils.data import Dataset, DataLoader

Download small dataset for training

In [ ]:
url = ("https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")

file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print('Total number of chars:', len(raw_text))
print(raw_text[:99])

# Preprocessing

## Text tokenization (manual implementation not necessary for the code, theoretical explanation)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# remove white spaces
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])


## Vocabulary creation (manual implementation not necessary for the code, theoretical explanation)

In [ ]:
# generate ordered set of unique tokens
all_words = sorted(set(preprocessed))
# add token for unkown words (not in the vocab) and for termination of documents when concatenation (inputs are concatenated and model should distinguish them)
all_words.extend(["<|endoftext|>", "<|unk|>"])
print(len(all_words))
print(all_words[:30])
print(all_words[-5:])


In [ ]:
# generate vocabulary
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  if i < 50:
    print(item)
  else:
    break

Class to convert from token to ID and vice-versa

In [ ]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    # inverse vocabulary permitting to map from int to token
    self.int_to_str = {integer:token for token, integer in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    # unkown tag if token not in vocab
    preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    # remove unnecessary spaces
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

Usage example

In [ ]:
# tokenizer = SimpleTokenizerV1(vocab)
# text = """"It's the last he painted, you know,"
#        Mrs. Gisburn said with pardonable pride."""

# ids = tokenizer.encode(text)
# print(ids)

# print(tokenizer.decode(ids))
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))


## Dataset class and dataloader (manual implementation not necessary for the code, theoretical explanation)
Through sliding window with parameters as context size (length) and stride

BPE tokenizer already implemented (instead of the basic ones implemented above)

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

In [ ]:
enc_sample = enc_text[:50]

In [ ]:
# elements in the input (window' size)
context_size = 4

dataset class -> each element in the dataset is a sequence of tokens, generating a dataset of successive strings. These sequences have a gap between each other of value 'stride'.

In [ ]:
class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+1+max_length]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]


Dataloader -> contains input batch and target batch because of the implementation of GPTDatasetV1 that generate the input ids and target ids

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = drop_last,
      num_workers = num_workers
  )

  return dataloader

Print first batch example

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

## Token embedding (manual implementation not necessary for the code, theoretical explanation)
Transform tokens IDs into embedding vectors (embedding can be optimized). Necessary since vectors are a continuous representation and neural networks use backward propagation for training. The embedding layer has dimension (vocab_size x embedding_dim), since for each word in the vocabulary we'll have an embedding vector of size embedding_dim. This embedding matrix is optimized during the training of the model as part of the training itself. Given then the token ID, it is possible to retrieve from the embedding matrix the embedding vector for that specific token.

In [ ]:
# size vocabulary BPE tokenizer
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

Instantiate data loader

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

# use embedding layer to embed the tokens inside the first batch
# retrieve embedding vectors given IDs
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

Add absolute embedding approach

In [ ]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# indices for positions 0,1,2 .. context_length - 1, i.e. 0, 1, 2 .. context_length - 1
# retrieve embedding vectors given IDs
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)


Add positional embeddings to basic embeddings = input embedding

In [ ]:
input_embeddings = token_embeddings + pos_embeddings

# Attention mechanism

## 1. Simplified version (not necessary for the code, theoretical explanation)

Embedding vectors of 'Your journey starts with one step'

In [ ]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
)


Computation attention score for a specific embedding vector (token) with respect to all the others -> dot product between selected token (embedded query token) and all the other embedding vectors

In [ ]:
# computing all attention score for all inputs, i.e. 6 values for each input
attn_scores = inputs @ inputs.T
print(attn_scores)


normalized attention scores, i.e. AS[2]/tot(AS) or with Softmax (more used since better managing of extreme values)

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print("Attention weights:", attn_weights)
print('All row sums:', attn_weights.sum(dim=-1))

Attention weights: sum of all input embedding multiplied with their attention score

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

## 2. Self-attention (with trainable parameters) (not necessary for the code, theoretical explanation)

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

Matrices (query, key, value)

In [ ]:
torch.manual_seed(123)
# requires_grad should be true if we really use them since they must be updated during training
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)


In [ ]:
# pytorch translate automatically x_2 to adjust the dimensions
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

Get all keys and values for all input

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

Attention score: dot product between query vector and key vector, i.e. attention score 2_2: query2.key2

In [ ]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

Attention score of all inputs with respect to input 2 (i.e. using query 2)

In [ ]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

Attention weights -> normalization and softmax -> divide attention scores by dividing them by the square root of the embedding dimension of the keys

In [ ]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2/d_k**0.5, dim=-1)
print(attn_weights_2)

Context vector for input 2 -> weighted sum over the value vectors

In [ ]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

Compact implementation using Python class for each input

In [ ]:
class SelfAttention_V1(nn.Module):
  def __init__(self, d_in, d_out, qkv_bias=False):
    # call superclass init since internal initialization logic should be executed for nn.Module
    super().__init__()
    # nn.Linear has optimized weight initialization scheme
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

  # x = all inputs
  def forward(self, x):
    keys = self.W_key(x)
    values = self.W_value(x)
    queries = self.W_query(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
    context_vec = attn_weights @ values
    return context_vec

How to use it (example)

In [ ]:
torch.manual_seed(789)
sa_v1 = SelfAttention_V1(d_in, d_out)
# returns the context vectors
print(sa_v1(inputs))

## 3. Causal attention (not necessary for the code, theoretical explanation)
Set to 0 all attention weights associated with the tokens after the selected one (lower triangular matrix)

In [ ]:
queries = sa_v1.W_query(inputs)
keys = sa_v1.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

Create lower triangular matrix

In [ ]:
context_length = attn_weights.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

Insert attention weights values

In [ ]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

Renormalize values to have sum = 1 for each row (divide each row values for the sum in each row)

In [ ]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums

print(masked_simple_norm)


Set 0s values to -inf -> softmax treat them as 0 probability -> more efficient masking trick

In [ ]:
# diagonal = 1 -> start from the diagonal above the middle one
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# put -inf to all values above lower triangular
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

Softmax for normalization

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

Implement dropout layer to avoid overfitting

In [ ]:
torch.manual_seed(123)
# in general dropout used: 0.2/0.3
dropout = torch.nn.Dropout(0.5)
print(dropout(attn_weights))

Add an input of 6 tokens, for a total of 2 inputs

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)
print(batch)

Self attention class with causal mask component and dropout

In [ ]:
class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()
    self.d_out = d_out
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)

    # mask has size context_length x context_length since we need to store the attention scores and attention weights for each token before the current one (lower triangular matrix)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length, diagonal=1)))

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys = self.W_key(x)
    values = self.W_value(x)
    queries = self.W_query(x)

    # transpose last two dimensions of keys to enable matrix multiplication -> from (batch_size, tokens_nr, embedding_dim) to (batch_size, embedding_dim, )
    attn_scores = queries @ keys.transpose(1,2)
    # access the mask above saved as buffer -> not optimized during backpropagation but available during the forward pass (often with masks)
    attn_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vec = attn_weights @ values
    return context_vec





## 4. Multi-head attention
Multiple queries, keys and values in parallel permits to compute different attention weights and attention scores. The resulting context vectors are then concatenated.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    # create the desired number of heads using the class above that generates a single causal attention head
    self.heads = nn.ModuleList([CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)])

  def forward(self, x):
    # head(x) call the forward method of the class CausalAttention and returns the context vectors for a single head
    return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    # the inputs are multiplied by the matrices Q,K,V generating the reduced q,v,k which have the same dimension of the output context vector.
    # For the parallel computation, q,v,k are split across the multiple heads, thus the dimension of q,v,k should be at least # heads
    assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.out_proj = nn.Linear(d_out, d_out)

    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    # view = reshape from [b, num_tokens, d_in] to [b, num_tokens, self.num_heads, self.head_dim] -> dimensions of k,q,v for each head
    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)

    # transpose in order to compute separately the results for each head, pass from [b, num_tokens, self.num_heads, self.head_dim] to [b, num_tokens, self.head_dim, self.num_heads]
    keys = keys.transpose(1, 2)
    queries = queries.transpose(1, 2)
    values = values.transpose(1, 2)

    attn_scores = queries @ keys.transpose(2,3)
    # not always context_sizes correspond to num_tokens (last batch, last input can have less tokens), thus cut
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
    attn_scores.masked_fill_(mask_bool, -torch.inf)

    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)

    # transpose again to pass from [b, num_tokens, self.head_dim, self.num_heads] to [b, self.head_dim, num_tokens, self.num_heads]
    # permit to concanenate easier the results of the different heads
    context_vec = (attn_weights @ values).transpose(1, 2)

    # combine head results
    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

    # raw concatenation is not enough for the models, thus this is used to refine the concatenation. Not mandatory but commonly used
    context_vec = self.out_proj(context_vec)
    return context_vec


In [ ]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

# LLM model
Combine different sections and implement the GPT model

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,   # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "drop_rate": 0.1,  # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

GPT dummy model

In [ ]:
class DummyGPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate"])
    # For now use DummyTransformerBlockas placeholder for actual transformer layers
    self.trf_blocks = nn.Sequential(*[DummyTransformerBlock(cfg)for _ in range(cfg["n_layers"])])

    # for now placeholder as normal layer
    self.final_norm = DummyLayerNorm(cfg["emb_dim"])
    # map embedding layer vocab_size x emb_dim into output layer emb_dim x vocab_size, such emb_dim is projected to vocab dimension, and applying a sofmax we can get the
    # next work that is the one with the highest probability
    self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    # lookup in the tok_emb for the indices given in in_idx
    tok_embeds = self.tok_emb(in_idx)
    # seq_len = context_len
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    # final results
    logits = self.out_head(x)
    return logits

Transformer blocks not implemented yet

In [ ]:
class DummyTransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()

  def forward(self, x):
    return x

Normalization not implemented yet

In [ ]:
class DummyLayerNorm(nn.Module):
  def __init__(self, normalized_shape, eps=1e-5):
    super().__init__()

  def forward(self, x):
    return x

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
print(batch)
batch = torch.stack(batch, dim=0)
print(batch)

In [ ]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
# each token has embedding size of 50'257 because is the number of tokens in the vocab, then we'll use a softmax to know which token with highest probability will be the next
print("Output shape:", logits.shape)
print(logits)

## 1. Normalization layer
Used to avoid explosion or vanishing of the gradient through standard deviation (subtract the mean and divide by square root of variance)

In [ ]:
class LayerNorm(nn.Module):
  def __init__(self, emb_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emb_dim))
    self.shift = nn.Parameter(torch.zeros(emb_dim))

  def forward(self, x):
      mean = x.mean(dim=-1, keepdim=True)
      var = x.var(dim=-1, keepdim=True, unbiased=False)
      norm_x = (x - mean) / torch.sqrt(var + self.eps)
      return self.scale * norm_x + self.shift

## 2. FNN with GELU
GELU with respect to ReLu is better for optimization.
The FNN layer permits the model to generalize and learn better the data: despite the input and output dimension is the same, internally this dimension is expanded, which permit the exporation of a richer representation space.

In [ ]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
                                GELU(),
                                nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),)

  def forward(self, x):
    return self.layers(x)

## 3. Shortcut connections
Added between different layers to improve training performance, since they avoid gradient vanishing by skipping some layers. How? Adding inpute values to the output of certain layers

In [ ]:
target = torch.tensor([[0.]])
print(target)

In [ ]:
class ExampleDeepNeuralNetwork(nn.Module):
  def __init__(self, layer_sizes, use_shortcut):
    super().__init__()
    self.use_shortcut = use_shortcut
    self.layers = nn.ModuleList([
      nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
      nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
      nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
      nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
      nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
    ])

  def forward(self, x):
    for layer in self.layers:
      layer_output = layer(x)
      if self.use_shortcut and x.shape == layer_output.shape:
        x = x + layer_output
      else:
        x = layer_output
    return x

In [ ]:
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123)
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=True
)

In [ ]:
def print_gradients(model, x):
  output = model(x)
  target = torch.tensor([[0.]])
  loss = nn.MSELoss()
  loss = loss(output, target)
  loss.backward()
  for name, param in model.named_parameters():
    if 'weight' in name:
      print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [ ]:
print_gradients(model_without_shortcut, sample_input)

## 4. Transformer block
Composed by multi-head attention, layer normalization, dropout, feed forward layers, GELU activation function

In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
        d_in=cfg["emb_dim"],
        d_out=cfg["emb_dim"],
        context_length=cfg["context_length"],
        num_heads=cfg["n_heads"],
        dropout=cfg["drop_rate"],
        qkv_bias=cfg["qkv_bias"]
    )
    self.ff = FeedForward(cfg)
    # normalize the embedding dimension
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

  def forward(self, x):
    # save input for attention shortcut
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    # shortcut for FNN
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    return x



## GPT architecture class

In [ ]:
class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate"])

    self.trf_blocks = nn.Sequential(
    *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(
        cfg["emb_dim"], cfg["vocab_size"], bias=False
    )

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

Run model

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Compute total number of trainable/trained parameters

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Generation of possibly multiple tokens, whose length depends on the user.Multiple iteration of the generation of a next token permit the model to generate meaninful complete sentences.

Approaches:
1. **softmax** -> the model returns everytime the token with the highest probability. The result is always the same every time the model is used.

2. **multinomial** -> the model returns most of the time the token with the highest probability, but not always. In this way, the originality of the output text is improved.

3. **temperature scaling** -> divide logitcs by a value greater than 0. Higher is the value of teh temperature, higher will be the originality. Too high values are not recommended (> 2/3), since we can have nonsensical output test

**top-k approach**: possibly used before temperature scaling or multinomial to avoid nonsensical text -> select top k tokens with highest probabilities after softmax, and set to -inf all other logits s.t. the resulting probability with the softmax is 0 for all the other tokens, then normalize to have all remaining probabilities summed up to 1 (automatically done using the softmax).


Simple version without text originality (used in the model traning)

In [ ]:
# idx -> input with size (batch, token ids)
def generate_text_simple(model, idx, max_new_tokens, context_size):
  for _ in range(max_new_tokens):
    # crops current context size to supported context size by the model -> if model supports size 5 and context size is 10, maintain last 5 tokens as context
    idx_cond = idx[:, -context_size:]
    with torch.no_grad():
      logits = model(idx_cond)

    # output of model is (batch, token, vocab), thus take last token
    logits = logits[:, -1, :]

    # temperature scaling

    # temperature = 0.5
    # logits = logits/temperature

    probas = torch.softmax(logits, dim=-1)

    # get idx of the next token that will be part of the next input given to the model
    idx_next = torch.argmax(probas, dim=-1, keepdim=True)
    # multinomial option -> take one sample and convert to tensor
    # idx_text = torch.multinomial(probas, num_sample=1).item()

    idx = torch.cat((idx, idx_next), dim=1)
  return idx


Complete version with text originality (not used in the model training)

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
  for _ in range(max_new_tokens):
    idx_cond = idx[:, -context_size:]
    with torch.no_grad():
      logits = model(idx_cond)
    logits = logits[:, -1, :]
    # choose if possible top k tokens
    if top_k is not None:
      top_logits, _ = torch.topk(logits, top_k)
      min_val = top_logits[:, -1]
      # set all other logits to -inf
      logits = torch.where(logits < min_val, torch.tensor(float('-inf')).to(logits.device),logits)

    if temperature > 0.0:
      logits = logits / temperature
      probs = torch.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
    else:
      idx_next = torch.argmax(logits, dim=-1, keepdim=True)

    if idx_next == eos_id:
      break

    idx = torch.cat((idx, idx_next), dim=1)
  return idx


In [ ]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
# add batch dimension (instead of (4), we get (1,4))
encoded_tensor = torch.tensor(encoded).unsqueeze(0)

# put model in eval mode to remove layers used only for training (e.g. dropout layers)
model.eval()

out = generate_text_simple(
  model=model,
  idx=encoded_tensor,
  max_new_tokens=6,
  context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

## Conversion from GPT output to token
The result of GPT model is a matrix. To get the predicted token, it is required to extract the last vector, apply the softmax to get probabilities, identify the index associated with the highest probability which is also the token ID, and get the token given its specific ID.

In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

# Pretraining

In [ ]:
# shorter context_length
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

Functions to convert from tokens to tokens IDs and opposite

In [ ]:
def text_to_token_ids(text, tokenizer):
  encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
  # insert batch dimension
  encoded_tensor = torch.tensor(encoded).unsqueeze(0)
  return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
  # remove batch dimension
  flat = token_ids.squeeze(0)
  return tokenizer.decode(flat.tolist())

Try functions

In [ ]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model = model,
    idx = text_to_token_ids(start_context, tokenizer),
    max_new_tokens = 10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## Loss computation without training
Load small dataset 'The verdict' to save time

In [ ]:
file_path = "the-verdict.txt"
with open(file_path, 'r', encoding='utf-8') as f:
  text_data = f.read()

get text length

In [ ]:
total_chars = len(text_data)
total_tok = tokenizer.encode(text_data)
print("Total characters:", total_chars)
print("Total tokens:", len(total_tok))

Split dataset into training set and validation set with ratio of 90% and 10%

In [ ]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

Data loader -> contains for each element input batch and target batch

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = drop_last,
      num_workers = num_workers
  )

  return dataloader

In [ ]:
train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

In [ ]:
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

Compute loss function for a single batch

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
  # transferring data to a device permit to transfer data to GPU
  input_batch = input_batch.to(device)
  target_batch = target_batch.to(device)
  logits = model(input_batch)
  # logits.flatten(0,1) -> merge dimensions 0 and 1 into a single one, from (batch_dim, nr_tokens, vocab_dim) to (batch_dim*nr_tokens, vocab_dim)
  loss = torch.nn.functional.cross_entropy(logits.flatten(0,1), target_batch.flatten())
  return loss


Compute loss for all batches of traning and validation set

In [ ]:
def calc_loss_loader(data_loader, model, device, num_batches = None):
  total_loss = 0.
  if len(data_loader) == 0:
    return float("nan")
  # if num_batches not specified
  elif num_batches is None:
    # dataloader returns data grouped by batches
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))
  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      # sum loss over batches
      total_loss += loss.item()
    else:
      break
  # average loss over all batches
  return total_loss/num_batches


Compute loss (high since no training performed yet)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
  train_loss = calc_loss_loader(train_loader, model, device)
  val_loss = calc_loss_loader(val_loader, model, device)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

## LLM Pretraining
traning:
1. iterate over each epoch (complete iteration over all input data)
2. iterate over each batch
3. reset loss gradients for each batch (batches are processed independently and their loss shouldn't be summed up).
4. compute loss on current batch (forward pass)
5. compute backpropagation (backward pass) to compute gradient of the loss with respect to the parameters (weights, biases)
6. update parameters based on the gradients get in the previous step
7. (print training and validation losses for tracking progress)

Evaluating the validation set within the training process permit to understand how the model perform on unseen data in order to use techniques to improve precision if needed (modify hyperparameter such as batch size, learning rate, etc. ) and can implement strategies such as early stopping to avoid a biased model on training data (overfitting). In addition spikes in validation data indicates issues in the training process

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_freq, eval_iter, start_context, tokenizer):
  # track losses and tokens seen
  train_losses, val_losses, track_tokens_seen = [], [], []
  # global_step initialized with -1 so that it can work as index
  tokens_seen, global_step = 0, -1

  for epoch in range(num_epochs):
    model.train()
    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      # compute loss gradients
      loss.backward()
      # update model parameters depending on loss gradients
      optimizer.step()
      tokens_seen += input_batch.numel()
      global_step += 1

      # eval_freq is the frequency on which we evaluate the model on a validation or test set (if 5, each 5 batches we evaluate the model), optional to see improvements
      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        track_tokens_seen.append(tokens_seen)

        print(f"Ep {epoch+1} (Step {global_step:06d}): "
              f"Train loss {train_loss:.3f}, "
              f"Val loss {val_loss:.3f}"
              )

    generate_and_print_sample(model, tokenizer, device, start_context)
  return train_losses, val_losses, track_tokens_seen




Evaluation: compute loss over training and evaluation sets ensuring model is in evaluation mode without dropout  

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
  model.eval()
  # not required in evaluation and save computational space
  with torch.no_grad():
    # compute loss for multiple batches to have more stable evaluation
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    # swap again to training mode for the training process in train_model_simple()
    model.train()
    return train_loss, val_loss

Evaluate the model by taking a text snippet and passing to the model

In [ ]:
# start_context = text snippet for evaluation purposes
def generate_and_print_sample(model, tokenizer, device, start_context):
  model.eval()
  # in the positional embedding weights the first shape is the context size
  context_size = model.pos_emb.weight.shape[0]
  encoded = text_to_token_ids(start_context, tokenizer).to(device)
  with torch.no_grad():
    # use the model in order to get the predicted next tokens starting from start_context
    token_ids = generate_text_simple(model=model, idx=encoded,max_new_tokens=50, context_size=context_size)
  decoded_text = token_ids_to_text(token_ids, tokenizer)
  print(decoded_text.replace("\n", " "))
  model.train()

Train the model

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(
     model.parameters(),
    lr=0.0004, weight_decay=0.1
)
num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer,
    device,num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer)

Plot training and validation loss -> the model overfit due to the small dataset and the absence of techniques to improve it, but this is a very basic training only for demonstration purposes

In [ ]:
def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
  fig, ax1 = plt.subplots(figsize=(5, 3))
  ax1.plot(epochs_seen, train_losses, label="Training loss")
  ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
  ax1.set_xlabel("Epochs")
  ax1.set_ylabel("Loss")
  ax1.legend(loc="upper right")
  ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
  ax2 = ax1.twiny()
  ax2.plot(tokens_seen, train_losses, alpha=0)
  ax2.set_xlabel("Tokens seen")
  fig.tight_layout()
  plt.show()

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

Save the model in order to not execute the training everytime (computational advantage) such that even if the session is closed the model will be still available. Two options:
1. save the model only -> if we don't perform any additional training (only inference)
2. save the model and the optimizer -> if additional training is performed, the optimizer contains important parameters that are needed in order to have a correct convergence

In [ ]:
# # save onyl the model
torch.save(model.state_dict(), "model.pth")

# # save the model and the optimizer
# torch.save({
#     "model_state_dict": model.state_dict(),
#     "optimizer_state_dict": optimizer.state_dict(),
#     },
#     "model_and_optimizer.pth"
# )

Load model (in general this is done in another notebook in order to not run the training again)

In [ ]:
checkpoint = torch.load("model.pth", map_location=device)
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# # if we save also the optimizer, this means we want to train it again thus set train state
# model.train();

## Load pretrained weights of GPT2
Since if the device is computationally limited the training is not effective, it is possible to load the weights of the GPT-2 models obtained during pretraining.

In [ ]:
# get gpt_download.py

url = (
    "https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch05/"
    "01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

Load GPT-2 architecture settings and parameters

In [ ]:
from gpt_download import download_and_load_gpt2
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

In [ ]:
print("Settings:", settings)
print("Parameter dictionary keys:", params.keys())

All configurations for GPT-2 models (small, medium, large, xl)

In [ ]:
model_configs = {
  "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
  "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
  "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
  "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

Update our previous config ('GPT_CONFIG_124M') with new parameters

In [ ]:
model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

NEW_CONFIG.update({"context_length": 1024})
# not used anymore but need to match the settings employed
NEW_CONFIG.update({"qkv_bias": True})

create new GPT instance with new parameters, however the initial weights are random

In [ ]:
gpt = GPTModel(NEW_CONFIG)
gpt.eval()

Function to check that left and right tensors have same shape (check if assignment of data is correct)

In [ ]:
def assign(left, right):
  if left.shape != right.shape:
    raise ValueError(f"Shape mismatch. Left: {left.shape}, " "Right: {right.shape}")
  return torch.nn.Parameter(torch.tensor(right))

Assign loaded parameters to the new model with the new configuration. Complex function and based on some guessing since GPT used different names. It is possible to load the pretrained GPT model without manually assign all parameters, but this is for demonstration purpose (i.e. model = GPT2Model.from_pretrained("gpt2"))

In [ ]:
def load_weights_into_gpt(gpt, params):
  # assign positional and token embeddings
  gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
  gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

  # iteration over all transformer blocks
  for b in range(len(params["blocks"])):
    # query, key and value components for self attention mechanism
    # split concatenated weights for q,k,v into 3 components q_w, k_w, v_w, transpose and assign
    q_w, k_w, v_w = np.split(
        (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
    gpt.trf_blocks[b].att.W_query.weight = assign(
        gpt.trf_blocks[b].att.W_query.weight, q_w.T)
    gpt.trf_blocks[b].att.W_key.weight = assign(
        gpt.trf_blocks[b].att.W_key.weight, k_w.T)
    gpt.trf_blocks[b].att.W_value.weight = assign(
        gpt.trf_blocks[b].att.W_value.weight, v_w.T)

    # same for biases
    q_b, k_b, v_b = np.split(
        (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
    gpt.trf_blocks[b].att.W_query.bias = assign(
        gpt.trf_blocks[b].att.W_query.bias, q_b)
    gpt.trf_blocks[b].att.W_key.bias = assign(
        gpt.trf_blocks[b].att.W_key.bias, k_b)
    gpt.trf_blocks[b].att.W_value.bias = assign(
        gpt.trf_blocks[b].att.W_value.bias, v_b)

    # output projection
    gpt.trf_blocks[b].att.out_proj.weight = assign(
        gpt.trf_blocks[b].att.out_proj.weight, params["blocks"][b]["attn"]["c_proj"]["w"].T)
    gpt.trf_blocks[b].att.out_proj.bias = assign(
        gpt.trf_blocks[b].att.out_proj.bias, params["blocks"][b]["attn"]["c_proj"]["b"])

    # feedforward nn
    gpt.trf_blocks[b].ff.layers[0].weight = assign(
        gpt.trf_blocks[b].ff.layers[0].weight, params["blocks"][b]["mlp"]["c_fc"]["w"].T)
    gpt.trf_blocks[b].ff.layers[0].bias = assign(
        gpt.trf_blocks[b].ff.layers[0].bias, params["blocks"][b]["mlp"]["c_fc"]["b"])
    gpt.trf_blocks[b].ff.layers[2].weight = assign(
        gpt.trf_blocks[b].ff.layers[2].weight, params["blocks"][b]["mlp"]["c_proj"]["w"].T)
    gpt.trf_blocks[b].ff.layers[2].bias = assign(
        gpt.trf_blocks[b].ff.layers[2].bias, params["blocks"][b]["mlp"]["c_proj"]["b"])

    # layer normalization, where scale='g' for gain and shift='b' for bias
    gpt.trf_blocks[b].norm1.scale = assign(
        gpt.trf_blocks[b].norm1.scale, params["blocks"][b]["ln_1"]["g"])
    gpt.trf_blocks[b].norm1.shift = assign(
        gpt.trf_blocks[b].norm1.shift, params["blocks"][b]["ln_1"]["b"])
    gpt.trf_blocks[b].norm2.scale = assign(
        gpt.trf_blocks[b].norm2.scale, params["blocks"][b]["ln_2"]["g"])
    gpt.trf_blocks[b].norm2.shift = assign(
        gpt.trf_blocks[b].norm2.shift, params["blocks"][b]["ln_2"]["b"])

  # final layer normalization and output head
  gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
  gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
  gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

In [ ]:
load_weights_into_gpt(gpt, params)
gpt.to(device)

Use model with loaded parameters to see token prediction and check that output text is coherent.

In [ ]:
torch.manual_seed(123)
token_ids = generate(
model=gpt,
idx=text_to_token_ids("Every effort moves you", tokenizer).to(device), max_new_tokens=25,
context_size=NEW_CONFIG["context_length"],
top_k=50,
temperature=1.5
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

# Fine-tuning (spam detection)
After the pretraining where the model learns how to perdict next tokens in a coherent way, LLMs can be fine-tuned to perform required task such as classification, question answering, sentiment analysis, text generation, etc. The fine-tuning can be divided into:
1. fine-tuning for classification: perform classification given an input such as spam detection, element recognition given images (tumors, object detection), text classification, etc. More specific than other fine-tuning since it is specialized in performing only the classification it saw only during the training phase

2. fine-tuning to follow instruction: fine-tune model such that given some instructions (in form of prompts) in natural language it is able to understand and execute the required different tasks. E.g. "**Translate in German** 'text to translate' ". Can perform different tasks

## Dataset preprocessing

Download dataset for spam detection

In [ ]:
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
# name of downloaded file
zip_path = "sms_spam_collection.zip"
# where the unzipped file is stored
extracted_path = "sms_spam_collection"
# create path pointing to sms_spam_collection appending SMSSpamCollection to the path -> after the unzip the dataset will be called SMSSpamCollection.tsv
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
  if data_file_path.exists():
    print(f"{data_file_path} already exists. Skipping download and extraction.")
    return
  # download file (write in zip_path the file read from url contained in 'response')
  with urllib.request.urlopen(url) as response:
    with open(zip_path, "wb") as out_file:
        out_file.write(response.read())
  # unzip file
  with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extracted_path)
  # add .tsv file extension
  original_file_path = Path(extracted_path) / "SMSSpamCollection"
  # add .tsv extension
  os.rename(original_file_path, data_file_path)
  print(f"File downloaded and saved as {data_file_path}")


In [ ]:
download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

Load .tsv file in a dataframe

In [ ]:
# /t since it it tab-separated (TSV) to know how to separate columns
df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"] )
df

In [ ]:
print(df["Label"].value_counts())

Simple dataset balancing since the two classes 'spam' and 'ham' have very different sizes -> done to avoid class bias and improve generalization (not performant with the small classes)

In [ ]:
def create_balanced_dataset(df):
  num_spam = df[df["Label"] == 'spam'].shape[0]
  # sample randomly num_spam number of ham data to balance the two classes. Ensure reproducibility with random_state
  ham_subset = df[df["Label"] == 'ham'].sample(num_spam, random_state=123)
  # concatenate new column of the ham subset with all spam
  balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
  return balanced_df

In [ ]:
balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Convert labels into integers 0 and 1

In [ ]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

Split data in dataset randomly in training set (70%), validation set (10%), test set (20%)

In [ ]:
def random_split(df, train_frac, validation_frac):
  # sample randomly data, since we take frac=1, we simply shuffle them. Drop old indices and reset them
  df = df.sample(frac=1, random_state=123).reset_index(drop=True)
  # get indices of split
  train_end = int(len(df) * train_frac)
  validation_end = train_end + int(len(df) * validation_frac)

  train_df = df[:train_end]
  validation_df = df[train_end:validation_end]
  test_df = df[validation_end:]

  return train_df, validation_df, test_df


In [ ]:
train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)

Save datasets as CSV

In [ ]:
train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

Create data loaders -> for the pretraining we had a sliding window that permit to had sentences of the same length, but in emails the lengths are different, thus:
1. truncate all messages to the length of the shortest one -> cheaper but there is the possibility of information loss if the shortest message is really short

2. add padding to all messages reaching the length of the longest one -> expensive but preserve information -> in this case we use this option with "<|endoftext|>" as padding token


In [ ]:
class SpamDataset(Dataset):
  def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
    self.data = pd.read_csv(csv_file)
    self.encoded_texts = [tokenizer.encode(text) for text in self.data["Text"]]
    if max_length is None:
      self.max_length = self._longest_encoded_length()
    else:
      self.max_length = max_length
    self.encoded_texts = [encoded_text[:self.max_length]for encoded_text in self.encoded_texts]
    self.encoded_texts = [encoded_text + [pad_token_id] *(self.max_length - len(encoded_text))for encoded_text in self.encoded_texts]

  def __getitem__(self, index):
    encoded = self.encoded_texts[index]
    label = self.data.iloc[index]["Label"]
    return (torch.tensor(encoded, dtype=torch.long), torch.tensor(label, dtype=torch.long))

  def __len__(self):
    return len(self.data)

  def _longest_encoded_length(self):
    max_length = 0
    for encoded_text in self.encoded_texts:
      encoded_length = len(encoded_text)
      if encoded_length > max_length:
        max_length = encoded_length
    return max_length


Pad train, test and validation sets

In [ ]:
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

In [ ]:
val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

In [ ]:
test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

In [ ]:
print(train_dataset.max_length)

Generate data loader -> each batch contains 8 input, where an input is an email text of 120 tokens and the label is 0 or 1 depending if spam or not

In [ ]:
num_workers = 0
batch_size = 8
torch.manual_seed(123)

In [ ]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

In [ ]:
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [ ]:
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

Check batch sizes if correct

In [ ]:
for input_batch, target_batch in train_loader:
  pass
print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Print total batches in each set

In [ ]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

## Model setup

### Model initialization

Same configuration as for unlabeled data

In [ ]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

In [ ]:
BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True
}

In [ ]:
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

In [ ]:
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

Download weights into gpt-2 model

In [ ]:
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval()

Check if model outputs coherent text, thus the parameters has been loaded correctly

In [ ]:
text_1 = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer),
    max_new_tokens=15,
    context_size=BASE_CONFIG["context_length"]
)
print(token_ids_to_text(token_ids, tokenizer))

### Classification head
Substitute the original output layer that maps to the vocabulary ('out_head') with a smaller output layer that maps to two classes only: 0 and 1 (not spam or spam). Without fine-tuning, with the new output layer the output of the model will be (batch_size, nr_tokens, vocab_dim), but the vocab_dim instead of being 50257 is now 2 (embedding dim) (for prediction of next token we take into consideration only the last token of the output).

We freeze the model for the fine-tuning in order to not update the parameters -> keep the pre-trained weights intact and only train the new layers or the final layer to adapt the model to a new task or dataset

In [ ]:
for param in model.parameters():
  param.requires_grad = False

Change final layer. This one has requires_grad = True automatically, thus this will be the only layer updated during fine-tuning. However despite the fact this should be sufficient, by empirical experiments fine-tuning additional layers improve the performance of the model

In [ ]:
torch.manual_seed(123)
num_classes = 2

model.out_head = torch.nn.Linear(
    # = 768, output dimension of transformer blocks
    in_features = BASE_CONFIG["emb_dim"],
    out_features = num_classes,
)

Make trainable last transformer block and final LayerNorm

In [ ]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

Consider only last token of the output for the classification. Why? Given the implementation of the attention mask that permit each token to have information about all preceeding tokens in the sequence, the last one is the one with most information. Using the softmax on these two values for each input text we get the probability to be spam or not spam, and we take the highest one as the correct class

In [ ]:
with torch.no_grad():
    outputs = model(inputs)

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape)
print("Last output token:", outputs[:, -1, :])

### Implement evaluation utilities
Compute classification loss and model accuracy (percentage of correct predictions)

In [ ]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
  model.eval()
  correct_predictions, num_examples = 0, 0
  if num_batches is None:
    num_batches = len(data_loader)
  else:
      num_batches = min(num_batches, len(data_loader))

  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      input_batch = input_batch.to(device)
      target_batch = target_batch.to(device)

      with torch.no_grad():
        logits = model(input_batch)[:, -1, :]
      predicted_labels = torch.argmax(logits, dim=-1)
      num_examples += predicted_labels.shape[0]

      correct_predictions += ((predicted_labels == target_batch).sum().item())
    else:
      break
  return correct_predictions / num_examples

function loss: cross-entropy for a single batch

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
  input_batch = input_batch.to(device)
  target_batch = target_batch.to(device)
  logits = model(input_batch)[:, -1, :]
  print(logits.shape, target_batch.shape)
  loss = torch.nn.functional.cross_entropy(logits, target_batch)
  return loss

loss for all batches

In [ ]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
  total_loss = 0.
  if len(data_loader) == 0:
    return float("nan")
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))
  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break
  return total_loss / num_batches

Initial loss

In [ ]:
with torch.no_grad():
  train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)

val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

## Model fine-tuning and usage